# Cas Kaggle

**Nerea de La Torre Veguillas 1669013**

**Helena Boneu Vidal 1673129**

Hem utilitzat el dataset **Harry Potter Dataset**.

In [2]:
import pandas as pd
import numpy as np

Carreguem el **dataset**:

Quan vam observar el datset per primer cop, vem veure que hi havien bastantes característiques dels personatges que no hi eren, per això vem decidir fer una cerca a internet i buscar la informació faltat. Tot i així, no vem poder aconseguir trobar totes les dades necessàries.

In [3]:
df = 'C:\\Users\\helen\\OneDrive\\Documentos\\3r Carrera\\1r semestre\\Aprenentatge Computacional\\Kaggle\\data\\harry-potter-dataset\\Characters.csv'

df = pd.read_csv(df, delimiter=';')
df.head(5)

,Id,Name,Gender,Job,House,Wand,Patronus,Species,Blood status,Hair colour,Eye colour,Loyalty,Skills,Birth,Death
0,1.0,Harry James Potter,Male,Student,Gryffindor,"11"" Holly phoenix feather",Stag,Human,Half-blood,Black,Bright green,Albus Dumbledore | Dumbledore's Army | Order o...,Parseltongue| Defence Against the Dark Arts | ...,31 July 1980,NaN
1,2.0,Ronald Bilius Weasley,Male,Student,Gryffindor,"12"" Ash unicorn tail hair",Jack Russell terrier,Human,Pure-blood,Red,Blue,Dumbledore's Army | Order of the Phoenix | Hog...,Wizard chess | Quidditch goalkeeping,1 March 1980,NaN
2,3.0,Hermione Jean Granger,Female,Student,Gryffindor,"10¾"" vine wood dragon heartstring",Otter,Human,Muggle-born,Brown,Brown,Dumbledore's Army | Order of the Phoenix | Hog...,Almost everything,"19 September, 1979",NaN
3,4.0,Albus Percival Wulfric Brian Dumbledore,Male,Headmaster,Gryffindor,"15"" Elder Thestral tail hair core",Phoenix,Human,Half-blood,Silver| formerly auburn,Blue,Dumbledore's Army | Order of the Phoenix | Hog...,Considered by many to be one of the most power...,Late August 1881,"30 June, 1997"
4,5.0,Rubeus Hagrid,Male,Keeper of Keys and Grounds | Professor of Care...,Gryffindor,"16"" Oak unknown core",NaN,Half-Human/Half-Giant,Part-Human (Half-giant),Black,Black,Albus Dumbledore | Order of the Phoenix | Hogw...,Resistant to stunning spells| above average st...,6 December 1928,NaN


In [4]:
print("Columnes amb nans:\n")
nan_columns = df.columns[df.isna().sum()>0].tolist()
print (df.loc[:,nan_columns].isna().sum()/len(df)*100)

Columnes amb nans:

Id               0.709220
Name             0.709220
Gender           0.709220
Job              5.673759
House           17.021277
Wand             0.709220
Patronus         5.673759
Species          0.709220
Blood status     4.964539
Hair colour      1.418440
Eye colour      12.765957
Loyalty          0.709220
Skills           5.673759
Birth            9.929078
Death           70.212766
dtype: float64


Als personatges que no son mags, podriem posar a les característiques 'House, Wand, Patronus' No?

També veiem que les característiques 'Id, Birth, Death' no són rellevants per a la nostra predicció. Per tant, les podriem eliminar del nostre dataset.

In [5]:
df = df.drop(columns=['Id', 'Birth', 'Death'])
print(df.keys())

Index(['Name', 'Gender', 'Job', 'House', 'Wand', 'Patronus', 'Species',
       'Blood status', 'Hair colour', 'Eye colour', 'Loyalty', 'Skills'],
      dtype='object')


A continuació mirem quin **tipus** de **dades** tenim.

In [6]:
print (df.dtypes)

Name            object
Gender          object
Job             object
House           object
Wand            object
Patronus        object
Species         object
Blood status    object
Hair colour     object
Eye colour      object
Loyalty         object
Skills          object
dtype: object
